In [1]:
from dotenv import load_dotenv
import streamlit as st
from PyPDF2 import PdfReader
load_dotenv()

True

In [2]:
class input_media:
    def __init__(self) -> None:
        load_dotenv()
        return
    def pdf(self):
        st.set_page_config(page_title="Upload pdf")
        st.header("Upload pdf")
        pdf = st.file_uploader("Upload your pdf", type='pdf')
        text = ""
        if pdf is not None:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text 
           

In [3]:
with open('state_of_the_union.txt', 'r', encoding='utf8') as f:
    loader = f.read()

In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [5]:
texts = loader.split('\n\n')

In [48]:
embeddings = HuggingFaceEmbeddings()

In [7]:
import numpy as np
# np.save('embeddings.npy', embeddings)
embeddings = np.load('embeddings.npy', allow_pickle=True)

c:\Users\ayush\OneDrive\Desktop\PROJECTS\Hackaton-QA-Bot\hackathonQA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))])

In [119]:
docsearch.similarity_search_with_score("What did the president say about Justice Breyer?")

[(Document(page_content='Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. ', metadata={'source': '262-pl'}),
  0.9261012077331543),
 (Document(page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. ', metadata={'source': '264-pl'}),
  0.9602278470993042),
 (Document(page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. ', metadata={'source': '263-pl'}),
  1.115667700767517),
 (Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My f

In [1]:
from huggingface_hub.inference_api import InferenceApi
import os

c:\Users\ayush\OneDrive\Desktop\PROJECTS\Hackaton-QA-Bot\hackathonQA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
inference = InferenceApi(repo_id="databricks/dolly-v2-12b", token=os.environ['HUGGINGFACEHUB_API_TOKEN'])
inputs = {"question":"Where is Hugging Face headquarters? strictly based on given context - Hugging Face is based in Brooklyn, New York. There is also an office in Paris, France."}
inference(inputs)

KeyboardInterrupt: 

In [121]:
%%time
#USING FAISS(Same results)
import numpy as np
import faiss
sentence_embeddings =  np.array(embeddings.embed_documents(texts))#time consuming step
d = len(sentence_embeddings[0])
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)
if not index.is_trained:
    index.train(sentence_embeddings)
index.add(sentence_embeddings)
k = 4
xq = np.array(embeddings.embed_query("What did the president say about Justice Breyer?"))
xq = np.expand_dims(xq,0)
D, I = index.search(xq, k)
print(I)

[[262 264 263   0]]
CPU times: total: 38.4 s
Wall time: 32.9 s
